In [ ]:
R1 = 'base_roster.json'
R2 = 'auto_roster_2019.json'
R1 = 'BBGM_League_5_1985_preseason.json'
R2 = 'BBGM_League_2_1985_preseason.json'
R1 = 'BBGM_League_3_2019_preseason.json'
R2 = 'BBGM_League_4_2019_preseason.json'
import json
import os
import sys
import fnmatch


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import scipy.stats

In [ ]:
def load_roster(filename):
    base = json.load(open(filename,'rb'))

    
    for i in range(len(base['players'])):
        player = base['players'][i]
        if 'name' in player and player['name'] !='':
            sname = base['players'][i]['name'].split()
            base['players'][i]['firstName'] = sname[0]
            base['players'][i]['lastName'] = ' '.join(sname[1:])
            del base['players'][i]['name']
    players = base['players']
    base['version'] = 32

    return players,base

In [ ]:
p1,b1 = load_roster(R1)
p2,b2 = load_roster(R2)

p1d = {_['firstName'] + ' ' + _['lastName']: _['ratings'][0] for _ in p1 if _['tid'] >=0}
p2d = {_['firstName'] + ' ' + _['lastName']: _['ratings'][0] for _ in p2 if _['tid'] >=0}

In [ ]:
df1 = pd.DataFrame(p1d).T.drop(['fuzz','skills','season','pos'],1).apply(pd.to_numeric)
df2 = pd.DataFrame(p2d).T.drop(['fuzz','skills','season','pos'],1).apply(pd.to_numeric)
merged = pd.merge(df1,df2,left_index=True,right_index=True)

In [ ]:
diffs = (df1 - df2).dropna(how='all').dropna(1,how='all')

In [ ]:
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

viridis = cm.get_cmap('viridis')
plt.figure(figsize=(10,8))

coeffs = []
for i,k in enumerate(df1.columns):
    c1 =  scipy.stats.spearmanr(merged[k+'_x'],merged[k+'_y'])
    coeffs.append((c1,k))
    
for i,v in enumerate(sorted(coeffs,reverse=True)):
    c1,k = v
    plt.subplot(4,5,i+1)
    plt.scatter(merged[k+'_x'],merged[k+'_y'],s=3,alpha=0.8,c=viridis(c1[0]-0.2))
    #c1 =  scipy.stats.spearmanr(merged[k+'_x'],merged[k+'_y'])
    #c2 =  scipy.stats.pearsonr(merged[k+'_x'],merged[k+'_y'])

    plt.xlabel('Alexnoob')
    plt.ylabel('Automatic')
    plt.title('{} ({:.2f})'.format(k,c1[0],c2[0]))
    plt.xlim(0,100)
    plt.ylim(0,100)
    plt.plot(np.arange(0,100),np.arange(0,100),c='k')
    plt.grid(True)
    plt.tight_layout()
plt.suptitle('Alexnoob vs. Automatic Rosters (2018-2019)',size=20)
plt.subplots_adjust(top=0.9)

In [ ]:
diffs.sort_values('pot',0,False)